# AutoKeras

ディープラーニングモデルを自動構築するPythonライブラリ

## データセットの読み込み(CIFAR-10)

In [1]:
from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 19s 0us/step


## モデルの構築

下記のように数行で学習を開始することはできるが，手元のPCではスペック不足により学習完了まで確認することができなかった．

In [3]:
import autokeras as ak

In [7]:
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=10,
    max_model_size=40000000,
    seed=1234)

clf.fit(x_train, y_train, epochs=10)

Trial 2 Complete [00h 44m 10s]
val_loss: 0.8444222807884216

Best val_loss So Far: 0.8444222807884216
Total elapsed time: 00h 45m 51s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|efficient         |resnet            
image_block_1/n...|True              |True              
image_block_1/a...|True              |True              
image_block_1/i...|True              |True              
image_block_1/i...|False             |True              
image_block_1/i...|0                 |0                 
image_block_1/i...|0                 |0                 
image_block_1/i...|0.1               |0.1               
image_block_1/i...|0                 |0                 
image_block_1/e...|True              |None              
image_block_1/e...|b7                |None              
image_block_1/e...|True              |None              
image_block_1/e...|True              |None              
classification_...|global_avg        |glo

RuntimeError: Too many consecutive oversized models.

## 困ったこと

### GPUが長時間後負荷状態になってしまう

AutoKerasでモデル学習中のGPUの状態は下図の通り．  
使用可能なメモリをすべて使い切り，高温状態となる．

![GPU Status](./figure/image_classifier-noconstraint-gpu_status.png)
    

1EPOCHあたり約30分要し，10EPOCHで5時間．  
このような探索を10Trial続けると2日以上，GPUの高負荷状態が続いてしまう為，手元のPCでは実行しないこととした．

![Training Log](./figure/image_classifier-noconstraint-log.png)


## パラメータ制約を設定するとエラー終了

```AutoKeras.ImageClasifier```の```max_model_size```でモデルの最大パラメータ数を設定可能だが，1,000,000個に設定すると探索すら開始されずにエラー終了する．

![Oversized model (1M)](./figure/image_classifier-constraint(1M)-log.png)

どうやら，約24Mパラメータは必要なようで，40Mパラメータで設定してみた．  
結果は，数モデル構築後に，上記と同様に，40Mパラメータ未満のモデルが探索されなくなり，エラー終了する．

![Oversized model(40M)](./figure/image_classifier-constraint(40M)-log.png)
